In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022aifinalexamp1/train.csv
/kaggle/input/2022aifinalexamp1/test.csv
/kaggle/input/2022aifinalexamp1/sample_submit.csv


In [2]:
#랜덤시드 고정
import random
import torch

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
#gpu 사용가능하면 사용하는 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
#데이터 로드
sample = pd.read_csv('/kaggle/input/2022aifinalexamp1/sample_submit.csv')
train = pd.read_csv('/kaggle/input/2022aifinalexamp1/train.csv')
test = pd.read_csv('/kaggle/input/2022aifinalexamp1/test.csv')

In [5]:
#학습데이터 확인
train.head()

,id,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,923,1027,True,2.2,False,0,False,63,0.8,102,...,152,714,1732,8,5,13,False,True,True,1
1,1268,1824,True,0.8,False,4,True,19,0.9,143,...,1055,1738,2381,14,1,5,True,False,False,2
2,1902,730,False,1.5,False,9,False,9,0.7,122,...,1371,1677,2787,10,8,7,True,False,True,2
3,88,668,False,0.5,True,0,False,3,0.1,155,...,666,1203,2746,9,5,8,True,True,True,2
4,41,1646,True,2.3,False,8,True,41,0.2,185,...,1725,1932,3339,18,10,19,True,True,False,3


In [6]:
#테스트데이터 확인
test.head()

,id,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,...,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,1860,1646,False,2.5,False,3,True,25,0.6,200,...,5,211,1608,686,8,6,11,True,True,False
1,353,1182,False,0.5,False,7,True,8,0.5,138,...,16,275,986,2563,19,17,19,True,False,False
2,1333,1972,False,2.9,False,9,False,14,0.4,196,...,18,293,952,1316,8,1,8,True,True,False
3,905,989,True,2.0,False,4,False,17,0.2,166,...,19,256,1394,3892,18,7,19,True,True,False
4,1289,615,True,0.5,True,7,False,58,0.5,130,...,8,1021,1958,1906,14,5,5,True,False,False


In [7]:
#데이터 정보 확인
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             1100 non-null   int64  
 1   battery_power  1100 non-null   int64  
 2   blue           1100 non-null   bool   
 3   clock_speed    1100 non-null   float64
 4   dual_sim       1100 non-null   bool   
 5   fc             1100 non-null   int64  
 6   four_g         1100 non-null   bool   
 7   int_memory     1100 non-null   int64  
 8   m_dep          1100 non-null   float64
 9   mobile_wt      1100 non-null   int64  
 10  n_cores        1100 non-null   int64  
 11  pc             1100 non-null   int64  
 12  px_height      1100 non-null   int64  
 13  px_width       1100 non-null   int64  
 14  ram            1100 non-null   int64  
 15  sc_h           1100 non-null   int64  
 16  sc_w           1100 non-null   int64  
 17  talk_time      1100 non-null   int64  
 18  three_g 

In [8]:
#데이터 전처리
xtrain = train.drop(['price_range'],axis=1)
ytrain = train['price_range']
xtest = test

In [9]:
#bool형식의 데이터 정수형으로 변환
from sklearn import preprocessing

label = ['blue','dual_sim','four_g','three_g','touch_screen','wifi']

for i in label:
    le = preprocessing.LabelEncoder()
    xtrain[i] = le.fit_transform(xtrain[i])
    xtest[i] = le.fit_transform(xtest[i])

In [10]:
#standardscaler를 이용한 데이터 전처리
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

xtrain_std = sc.fit_transform(xtrain)
xtest_std = sc.transform(xtest)

In [11]:
#텐서로 변환
X_train = torch.FloatTensor(np.array(xtrain_std)).to(device)
y_train = torch.LongTensor(np.array(ytrain)).to(device)
X_test = torch.FloatTensor(np.array(xtest_std)).to(device)

In [12]:
X_train.shape

torch.Size([1100, 21])

In [13]:
#데이터셋 선언과 데이터 로더 선언
from torch.utils.data import DataLoader, TensorDataset

dataset = TensorDataset(X_train, y_train)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

In [14]:
#모델 정의
import torch.nn as nn

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(21, 64, bias=True)
        self.linear2 = nn.Linear(64, 64, bias=True)
        self.linear3 = nn.Linear(64, 64, bias=True)
        self.linear4 = nn.Linear(64, 4, bias=True)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        
        nn.init.xavier_uniform_(self.linear1.weight)
        nn.init.xavier_uniform_(self.linear2.weight)
        nn.init.xavier_uniform_(self.linear3.weight)
        nn.init.xavier_uniform_(self.linear4.weight)
        
    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear3(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear4(x)
        
        return x

In [15]:
#모델 선언
model = Model().to(device)

In [16]:
#optimizer 정의
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
#Loss 정의 (다중분류문제이므로 CrossEntropyLoss)
loss = nn.CrossEntropyLoss()

In [17]:
#모델 학습
epochs = 100
model.train()
for epoch in range(epochs + 1):
    costs = []
    for x, y in dataloader:
        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(x)
        cost = loss(hypothesis, y)
        costs.append(cost.item())
        cost.backward()
        optimizer.step()
    
    if epoch % 10 == 0:
        average = np.mean(costs)
        print(f"epoch : {epoch}, cost : {average}")

epoch : 0, cost : 1.4214350912306044
epoch : 10, cost : 0.6649588313367631
epoch : 20, cost : 0.43501832915676963
epoch : 30, cost : 0.32627373933792114
epoch : 40, cost : 0.26957664142052334
epoch : 50, cost : 0.22381996115048727
epoch : 60, cost : 0.2118439194228914
epoch : 70, cost : 0.16485934828718504
epoch : 80, cost : 0.1472622269971503
epoch : 90, cost : 0.1360055154396428
epoch : 100, cost : 0.12558046045402685


In [18]:
#테스트 데이터를 이용한 모델 테스트
model.eval()
with torch.no_grad():
    hypothesis = model(X_test)
    predict = torch.argmax(hypothesis, dim=1)

In [19]:
#결과값을 csv파일로 변환
sample['price_range'] = predict.cpu().detach().numpy()
sample.to_csv('submit.csv',index=False)